In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import csv

In [ ]:
execution_path=os.getcwd() #Gets current working directory of this notebook

## Function to save frames of videos

In [ ]:
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(os.path.join(execution_path,"image"+str(count)+".jpg"), image)     # save frame as JPG file
    return hasFrames

## Loop to save frames of videos & Compute frame by frame similarity score

In [ ]:
for i in range (1,11,1):
    execution_path=os.getcwd()
    if i<=5:
        os.mkdir(os.path.join(execution_path,"abnormal_"+str(i)))
        execution_path=os.path.join(execution_path,"Videos SP Cup", "abnormal_"+str(i))
    else:
        os.mkdir(os.path.join(execution_path,"normal_"+str(i-5)))
        execution_path=os.path.join(execution_path, "Videos SP Cup", "normal_"+str(i-5))
    video_path=execution_path+".avi"
    print(video_path)
    vidcap = cv2.VideoCapture(video_path)
    sec = 0
    frameRate = 0.05 #Captures image in each 0.5 seconds
    count=1
    if i>=6:
        execution_path=os.path.join(os.getcwd(),"normal_"+str(i-5))
    else:
        execution_path=os.path.join(os.getcwd(),"abnormal_"+str(i))
    success = getFrame(sec)
    while success:
        count = count + 1
        sec = sec + frameRate
        sec = round(sec, 2)
        success = getFrame(sec)
    ct=1
    with open('mycsv'+str(i)+'.csv','w',newline='') as f:
        writer=csv.writer(f)
        writer.writerow(['percentage']) 
        while ct<count-1:
            original = cv2.imread(os.path.join(execution_path , "image"+str(ct)+".jpg"))
            duplicate = cv2.imread(os.path.join(execution_path , "image"+str(ct+1)+".jpg"))
            ct=ct+1
            #Computation of frame by frame similarity score
            difference = cv2.subtract(original, duplicate)
            b, g, r = cv2.split(difference)
            orb = cv2.ORB_create()
            kp_1, desc_1 = orb.detectAndCompute(original, None)
            kp_2, desc_2 = orb.detectAndCompute(duplicate, None)
            bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
            matches = bf.match(desc_1,desc_2)
            matches = sorted(matches, key = lambda x:x.distance)
            img3 = cv2.drawMatches(original,kp_1,duplicate,kp_2,matches[:10],None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            matches = bf.match(desc_1,desc_2)
            good_points = []
            ratio = 40
            for m in matches:
                if m.distance < ratio:
                    good_points.append(m)
            result = cv2.drawMatches(original, kp_1, duplicate, kp_2, good_points, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            number_keypoints = 0
            if len(kp_1) <= len(kp_2):
                number_keypoints = len(kp_1)
            else:
                number_keypoints = len(kp_2)
#           print("Keypoints 1ST Image: " + str(len(kp_1)))
#           print("Keypoints 2ND Image: " + str(len(kp_2)))
#           print("GOOD Matches:", len(good_points))
#           print("How good it's the match: ", len(good_points) / number_keypoints * 100, "%")
            perc=len(good_points) / number_keypoints * 100          
            writer.writerow([str(perc)])